In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import mean_absolute_error

data = pd.read_excel(r"I:\1\DataSet.xlsx")

X = data.drop(columns=["V"])
y = data["V"]  

imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

X_imputed[np.isinf(X_imputed)] = np.nan
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X_imputed)

X = pd.DataFrame(X_imputed, columns=X.columns)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.3)

model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mean_absolute_error'])

history = model.fit(X_train, y_train, validation_split=0.2,epochs=300, batch_size=16, verbose=1)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R^2: {r2}")

mae = mean_absolute_error(y_test, y_pred)

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test, y_pred.flatten())

print(f"MAE: {mae}")
print(f"MAPE: {mape}")

results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})
metrics_df = pd.DataFrame({'RMSE': [rmse], 'R^2': [r2], 'MAE': [mae], 'MAPE': [mape]})

with pd.ExcelWriter(r'I:\1\CNN.xlsx') as writer:
    results_df.to_excel(writer, sheet_name='Predicted results', index=False)
    metrics_df.to_excel(writer, sheet_name='Performance Indicators', index=False)

class R2Callback(Callback):
    def __init__(self, train_data, val_data):
        super().__init__()
        self.train_data = train_data
        self.val_data = val_data

    def on_train_begin(self, logs=None):
        self.r2s_train = []
        self.r2s_val = []

    def on_epoch_end(self, epoch, logs=None):
        y_pred_train = self.model.predict(self.train_data[0])
        r2_train = r2_score(self.train_data[1], y_pred_train)
        self.r2s_train.append(r2_train)

        y_pred_val = self.model.predict(self.val_data[0])
        r2_val = r2_score(self.val_data[1], y_pred_val)
        self.r2s_val.append(r2_val)
        
        print(f"Epoch {epoch+1}: train R2: {r2_train:.4f}, val R2: {r2_val:.4f}")

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

r2_callback = R2Callback(train_data=(X_train, y_train), val_data=(X_val, y_val))

history = model.fit(X_train, y_train, epochs=1000, verbose=1, validation_data=(X_val, y_val), callbacks=[r2_callback])

plt.figure(figsize=(10, 5))
plt.plot(r2_callback.r2s_train, label='Train R2')
plt.plot(r2_callback.r2s_val, label='Validation R2')
plt.title('R2 Score over Epochs')
plt.xlabel('Epoch')
plt.ylabel('R2 Score')
plt.legend()
plt.show()

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(np.sqrt(history.history['loss']), label='Train RMSE')
plt.plot(np.sqrt(history.history['val_loss']), label='Validation RMSE')
plt.title('RMSE Over Epochs')
plt.legend()
plt.show()

In [ ]:
from sklearn.inspection import permutation_importance

def evaluate_model(X, y, model):
    X_reshaped = X.reshape(X.shape[0], X.shape[1], 1)
    preds = model.predict(X_reshaped)
    return preds

feature_names = X.columns

original_performance = r2_score(y_test, model.predict(X_test))

results = []
for feature in range(X_train.shape[1]):  
    X_temp = np.copy(X_test)
    np.random.shuffle(X_temp[:, feature, 0])
    shuffled_performance = r2_score(y_test, evaluate_model(X_temp, y_test, model))
    importance = original_performance - shuffled_performance
    results.append(importance)

importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': results})

importance_df_sorted = importance_df.sort_values(by='Importance', ascending=False)

plt.barh(importance_df_sorted['Feature'], importance_df_sorted['Importance'])  
plt.title('Permutation Feature Importance')
plt.ylabel('Features') 
plt.xlabel('Importance')  
plt.show()

importance_df.to_excel(r'I:\1\Feature Importance.xlsx', index=False)

In [ ]:
new_data = pd.read_excel(r'I:\1\Data for prediction.xlsx')

X_new = new_data.drop(columns=["V"])

X_new_imputed = imputer.transform(X_new)
X_new_imputed[np.isinf(X_new_imputed)] = np.nan
X_new_imputed = imputer.transform(X_new_imputed)
X_new = pd.DataFrame(X_new_imputed, columns=X_new.columns)
X_new_scaled = scaler.transform(X_new)

X_new_reshaped = X_new_scaled.reshape(X_new_scaled.shape[0], X_new_scaled.shape[1], 1)

y_new_pred = model.predict(X_new_reshaped)

new_results_df = pd.DataFrame({'Predicted': y_new_pred.flatten()})
with pd.ExcelWriter(r'I:\1\CNN-new results.xlsx') as writer:
    new_results_df.to_excel(writer, sheet_name='Predicted results', index=False)